In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


2.8284271247461903


In [2]:
with open('sounds/wavToTag.txt') as f:
    labels = array([l.replace('\n', '') for l in f.readlines()])
    
print(set(labels))

{'stade', 'cocacola', 'zidane', 'biere', 'chaussure', 'beckham', 'ballon', 'jeuvideo', 'girondins', 'sofoot', 'manette', 'gants'}


In [4]:
len(labels)

245

In [5]:
import librosa as lb

In [6]:
mfccs = {}

for i in range(len(labels)):
    # sr = sample rate
    y, sr = lb.load('sounds/{}.wav' . format(i))
    mfcc = lb.feature.mfcc(y, sr, n_mfcc=13)
    mfccs[i] = mfcc.T

In [13]:
# Generate Training and Testing data

def generate_train_test_set(P):
    train = []
    test = []

    for s in set(labels):
        all = find(labels == s)
        shuffle(all)
        train += all[:-P].tolist()
        test += all[-P:].tolist()
        
    return train, test

In [14]:
from dtw import dtw

# We use DP to speed up multiple tests
D = ones((len(labels), len(labels))) * -1

def cross_validation(train, test):
    score = 0.0

    for i in test:
        x = mfccs[i]

        dmin, jmin = inf, -1
        for j in train:
            y = mfccs[j]
            
            d = D[i, j]
            if d == -1:
                d, _, _, _ = dtw(x, y, dist=lambda x, y: norm(x - y, ord=1))
                D[i, j] = d                

            if d < dmin:
                dmin = d
                jmin = j

        score += 1.0 if (labels[i] == labels[jmin]) else 0.0
        
    return score / len(test)

In [15]:
train, test = generate_train_test_set(P=1)
rec_rate = cross_validation(train, test)
print('Recognition rate {}%'.format(100. * rec_rate))

Recognition rate 66.66666666666666%


In [ ]:
P = arange(1, 10)
N = 5

rec = []

for p in P:
    r = [cross_validation(*generate_train_test_set(p)) for _ in range(N)]
    rec.append(r)
    
rec = array(rec)
rec = rec.reshape((N, -1))

errorbar(P - 0.5, mean(rec, axis=0), yerr=std(rec, axis=0))
xticks(P - 0.5, P)
ylim(0, 1)